# Test LLVM AST Notebook

## Author: Yiannis Charalambous


In [1]:
import os
from clang.cindex import Config
import clang.native
import clang.cindex
import sys
from typing import NamedTuple


In [2]:
# Connect the Python API of Clang to the libclang.so file bundled in the libclang PyPI package.
Config.library_file = os.path.join(
    os.path.dirname(clang.native.__file__),
    "libclang.so",
)

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
FILE = "../samples/threading.c"


def get_declarations_local(root: clang.cindex.Cursor) -> list[clang.cindex.Cursor]:
    declarations: list[clang.cindex.Cursor] = []
    declarations_raw: set[str] = {}
    tokens: list[clang.cindex.Token] = []
    # Scan all direct symbols in root.
    for child in root.get_children():
        # print(f"Scanning: {child.spelling}")
        node: clang.cindex.Cursor = child
        kind: clang.cindex.CursorKind = node.kind
        # Check if it is actually from the file.
        if (
            kind.is_declaration()
            and node.storage_class == clang.cindex.StorageClass.NONE
        ):
            print(
                f"Found {node.spelling} [line={node.location.line}, col={node.location.column}]"
            )
            tokens: clang.cindex.Token = node.get_tokens()
            for token in tokens:
                print(f"  Token {token.spelling} {token.kind}")
            loc: clang.cindex.SourceRange = node.extent
            end: clang.cindex.SourceLocation = loc.end
            start: clang.cindex.SourceLocation = loc.start
            print(
                f"Start: {start.offset}, End: {end.offset}, Range: {end.offset - start.offset}"
            )
            print()
            declarations.append(node)
    return declarations


index: clang.cindex.Index = clang.cindex.Index.create()
tu: clang.cindex.TranslationUnit = index.parse(FILE)
root: clang.cindex.Cursor = tu.cursor
declarations: clang.cindex.Cursor = get_declarations_local(root)

print(f"Total {len(declarations)}")


Found a [line=4, col=5]
  Token int TokenKind.KEYWORD
  Token a TokenKind.IDENTIFIER
Start: 42, End: 47, Range: 5

Found b [line=4, col=8]
  Token int TokenKind.KEYWORD
  Token a TokenKind.IDENTIFIER
  Token , TokenKind.PUNCTUATION
  Token b TokenKind.IDENTIFIER
Start: 42, End: 50, Range: 8

Found __VERIFIER_atomic_acquire [line=5, col=6]
  Token void TokenKind.KEYWORD
  Token __VERIFIER_atomic_acquire TokenKind.IDENTIFIER
  Token ( TokenKind.PUNCTUATION
  Token void TokenKind.KEYWORD
  Token ) TokenKind.PUNCTUATION
  Token { TokenKind.PUNCTUATION
  Token __VERIFIER_assume TokenKind.IDENTIFIER
  Token ( TokenKind.PUNCTUATION
  Token a TokenKind.IDENTIFIER
  Token == TokenKind.PUNCTUATION
  Token 0 TokenKind.LITERAL
  Token ) TokenKind.PUNCTUATION
  Token ; TokenKind.PUNCTUATION
  Token a TokenKind.IDENTIFIER
  Token = TokenKind.PUNCTUATION
  Token 1 TokenKind.LITERAL
  Token ; TokenKind.PUNCTUATION
  Token } TokenKind.PUNCTUATION
Start: 52, End: 134, Range: 82

Found c [line=10, col=7]

## Reversing Reach AST To Source Code

The only issue I have found, multiple declarations in one statement need to be recognized and the nodes combined:

```c
int a, b;
```


In [4]:
with open(FILE) as file:
    source_code: str = file.read()


def get_node_source_code(source_code: str, node: clang.cindex.Cursor) -> str:
    loc: clang.cindex.SourceRange = node.extent
    start: clang.cindex.SourceLocation = loc.start
    end: clang.cindex.SourceLocation = loc.end
    return source_code[start.offset : end.offset]


for node in declarations:
    print(f"Code for {node.displayname}:")
    print("```")
    print(get_node_source_code(source_code, node))
    print("```")
    print()


Code for a:
```
int a
```

Code for b:
```
int a, b
```

Code for __VERIFIER_atomic_acquire():
```
void __VERIFIER_atomic_acquire(void)
{
    __VERIFIER_assume(a == 0);
    a = 1;
}
```

Code for c(void *):
```
void *c(void *arg)
{
    ;
    __VERIFIER_atomic_acquire();
    b = 1;
    return NULL;
}
```

Code for d:
```
pthread_t d
```

Code for main():
```
int main()
{
    pthread_create(&d, 0, c, 0);
    __VERIFIER_atomic_acquire();
    if (!b)
        assert(0);
    return 0;
}
```



# Test Code


In [5]:
from esbmc_ai.frontend.ast import ClangAST


In [6]:
file = "../samples/threading.c"
cast = ClangAST(file)
functions = cast.get_fn_decl()

for fn in functions:
    print(str(fn) + "\n")
    # Seems like different cursors have the same translation unit...
    print(fn.cursor)
    print(fn.cursor.translation_unit)


__VERIFIER_atomic_acquire()

c(arg: void *)

main()



# Test Code 2


In [7]:
file = "./samples/typedefs.c"
cast = ClangAST(file)
functions = cast.get_type_decl()

for fn in functions:
    print(fn)


struct linear {value: int}
typedef (LinearTypeDef) struct linear {struct linear: struct linear}
Point {x: int, y: int}
typedef (Point) Point {Point: Point}
enum Types {ONE: int, TWO: int, THREE: int}
typedef (Typest) enum Types {Types: enum Types}
union Combines {a: int, b: int, c: int}
typedef (CombinesTypeDef) union Combines {union Combines: union Combines}


In [8]:
file = "./samples/typedefs.c"
cast: ClangAST = ClangAST(file)
includes = cast.get_include_directives()

for include in includes:
    print(include)


#include "/usr/include/stdlib.h"
#include "/usr/include/assert.h"
